In [1]:
# IMporting SQL libraries
from pyspark.sql.types import *
# from pyspark.sql.functions import * 
from pyspark.sql.functions import sum as _sum

In [2]:
# Reading files from createed table
csv = spark.sql('select * from azureData4')

In [3]:
csv.createTempView('mytable3')

In [4]:
csv.columns

Out[4]: ['processed_text', 'y']

In [5]:
spark.sql('select count(*) from mytable3').show()

+--------+
count(1)|
+--------+
 33183|
+--------+

In [6]:
# IMporting libraries
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="processed_text", outputCol="words", pattern="\\s+")

# stop words
add_stopwords = ["http","https","amp","rt","t","c","can", # standard stop words
     "#keithlamontscott","#charlotteprotest","#charlotteriots","#keithscott"] # keywords used to pull data)
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
# Here we created bag of words as 10000 beecause more than that will take hours to compute and sometimes fails to.
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [7]:
# Creating pipeline of basic data cleaning and creating dataset with all features.
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(csv)
# Transform dataset with new pipelined features.
dataset = pipelineFit.transform(csv)

In [8]:
dataset.dtypes

Out[9]: [('processed_text', 'string'),
 ('y', 'int'),
 ('words', 'array<string>'),
 ('filtered', 'array<string>'),
 ('features', 'vector')]

In [9]:
# Implementing logistic regression

In [10]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 23269
Test Dataset Count: 9914

In [11]:
trainingData.show()

+--------------------+---+--------------------+--------------------+--------------------+
 processed_text| y| words| filtered| features|
+--------------------+---+--------------------+--------------------+--------------------+
1 deadline compli...| 0|[1, deadline, com...|[1, deadline, com...|(10000,[2,3,4,5,8...|
1 introduction tr...| 1|[1, introduction,...|[1, introduction,...|(10000,[0,1,2,5,6...|
1,4-glcnac- ␤ pre...| 1|[1,4-glcnac-, ␤, ...|[1,4-glcnac-, ␤, ...|(10000,[0,1,2,3,4...|
10 case 23.81 urt...| 0|[10, case, 23.81,...|[10, case, 23.81,...|(10000,[1,2,3,4,5...|
114 catalytic hyd...| 1|[114, catalytic, ...|[114, catalytic, ...|(10000,[0,1,2,5,6...|
120 year pass wal...| 1|[120, year, pass,...|[120, year, pass,...|(10000,[0,1,2,3,4...|
17 160 type modif...| 1|[17, 160, type, m...|[17, 160, type, m...|(10000,[0,2,3,4,6...|
170 million chron...| 0|[170, million, ch...|[170, million, ch...|(10000,[0,1,2,5,7...|
170 million peopl...| 1|[170, million, pe...|[170, million, pe...|(10000,[0,1,2,5,6...|
1918 world war wi...| 0|[1918, world, war...|[1918, world, war...|(10000,[1,2,3,4,5...|
1959 richard feyn...| 1|[1959, richard, f...|[1959, richard, f...|(10000,[0,1,2,3,4...|
1980 average new ...| 0|[1980, average, n...|[1980, average, n...|(10000,[1,2,5,6,7...|
1980s world healt...| 0|[1980s, world, he...|[1980s, world, he...|(10000,[1,2,5,6,7...|
1995 haemophilus ...| 1|[1995, haemophilu...|[1995, haemophilu...|(10000,[0,1,2,7,8...|
1997 highly patho...| 1|[1997, highly, pa...|[1997, highly, pa...|(10000,[0,1,2,3,4...|
1997 study report...| 1|[1997, study, rep...|[1997, study, rep...|(10000,[0,1,2,3,4...|
1998 world health...| 0|[1998, world, hea...|[1998, world, hea...|(10000,[2,5,7,8,9...|
20 year alphaviru...| 1|[20, year, alphav...|[20, year, alphav...|(10000,[0,1,2,3,4...|
200 different typ...| 1|[200, different, ...|[200, different, ...|(10000,[0,1,2,3,4...|
2000-year medicin...| 1|[2000-year, medic...|[2000-year, medic...|(10000,[0,1,2,3,4...|
+--------------------+---+--------------------+--------------------+--------------------+
only showing top 20 rows

In [12]:
trainingData.dtypes

Out[145]: [('processed_text', 'string'),
 ('y', 'int'),
 ('words', 'array<string>'),
 ('filtered', 'array<string>'),
 ('features', 'vector')]

In [13]:
from pyspark.ml.classification import LogisticRegression
# Build the model
lr = LogisticRegression(labelCol='y', maxIter=20, regParam=0.3, elasticNetParam=0, family = "binomial")

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [14]:
trainingSummary = lrModel.summary

In [15]:
import matplotlib.pyplot as plt
objectiveHistory = trainingSummary.objectiveHistory
plt.plot(objectiveHistory)
plt.ylabel('Objective Function')
plt.xlabel('Iteration')
plt.show()

In [16]:
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

areaUnderROC: 0.9979946234942196

In [17]:
import matplotlib.pyplot as plt
import numpy as np

beta = np.sort(lrModel.coefficients)

plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

In [18]:
predictions = lrModel.transform(testData)

In [19]:
from pyspark.sql.functions import col
see = predictions.select('y', col('y').alias('label'), 'rawPrediction')

In [20]:
see.show()

+---+-----+--------------------+
 y|label| rawPrediction|
+---+-----+--------------------+
 0| 0|[5.85770883288753...|
 1| 1|[-3.7477454111755...|
 1| 1|[-5.2221739289534...|
 1| 1|[-4.1522020226677...|
 1| 1|[-0.9887836907868...|
 1| 1|[-1.9845441788294...|
 1| 1|[-0.5578657853123...|
 0| 0|[1.39997977579923...|
 1| 1|[-6.2685294033022...|
 1| 1|[-2.6162621398216...|
 0| 0|[7.99472341814452...|
 1| 1|[-4.0581854305074...|
 0| 0|[4.25756155042867...|
 0| 0|[6.77162591263091...|
 1| 1|[-4.4712351957883...|
 1| 1|[-2.9864624971901...|
 0| 0|[4.22158594445307...|
 0| 0|[2.99727677277623...|
 1| 1|[-4.3020059850553...|
 1| 1|[-4.3119701313827...|
+---+-----+--------------------+
only showing top 20 rows

In [21]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(see, {evaluator.metricName: "areaUnderROC"})))

Test: Area Under ROC: 0.9949234529734854

In [22]:
We are using decision tree classifier

In [23]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 23269
Test Dataset Count: 9914

In [24]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="y", featuresCol="features", maxDepth=3)
# Only for cli
# dt = DecisionTreeClassifier(labelCol="y", featuresCol="features")
# Train model with Training Data
dtModel = dt.fit(trainingData)

In [25]:
# Evaluate model
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = dtModel.transform(testData)

In [26]:
predictions.show()

+--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+----------+
 processed_text| y| words| filtered| features| rawPrediction| probability|prediction|
+--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+----------+
100,000 patient a...| 0|[100,000, patient...|[100,000, patient...|(10000,[2,3,4,7,8...|[7693.0,848.0]|[0.90071420208406...| 0.0|
14 species canadi...| 1|[14, species, can...|[14, species, can...|(10000,[1,2,5,7,8...| [535.0,712.0]|[0.42902967121090...| 1.0|
14.69 60.02 propo...| 1|[14.69, 60.02, pr...|[14.69, 60.02, pr...|(10000,[1,2,5,6,7...|[748.0,9253.0]|[0.07479252074792...| 1.0|
15 year ago opden...| 1|[15, year, ago, o...|[15, year, ago, o...|(10000,[0,1,2,6,7...|[748.0,9253.0]|[0.07479252074792...| 1.0|
1970s tomisaku ka...| 1|[1970s, tomisaku,...|[1970s, tomisaku,...|(10000,[0,2,3,4,5...|[748.0,9253.0]|[0.07479252074792...| 1.0|
1990 computer use...| 1|[1990, computer, ...|[1990, computer, ...|(10000,[0,2,6,7,8...|[748.0,9253.0]|[0.07479252074792...| 1.0|
1992 laboratory p...| 1|[1992, laboratory...|[1992, laboratory...|(10000,[0,1,2,5,6...| [185.0,564.0]|[0.24699599465954...| 1.0|
1995 ﻿1 pilot iss...| 0|[1995, ﻿1, pilot,...|[1995, ﻿1, pilot,...|(10000,[3,4,5,8,9...|[7693.0,848.0]|[0.90071420208406...| 0.0|
2 billion people ...| 1|[2, billion, peop...|[2, billion, peop...|(10000,[0,1,2,3,4...|[748.0,9253.0]|[0.07479252074792...| 1.0|
2 million people ...| 1|[2, million, peop...|[2, million, peop...|(10000,[0,1,2,5,7...| [535.0,712.0]|[0.42902967121090...| 1.0|
2003 2 million ca...| 0|[2003, 2, million...|[2003, 2, million...|(10000,[1,2,5,7,8...|[7693.0,848.0]|[0.90071420208406...| 0.0|
2003 coronavirus ...| 1|[2003, coronaviru...|[2003, coronaviru...|(10000,[0,1,2,3,4...|[748.0,9253.0]|[0.07479252074792...| 1.0|
2003 hong kong pr...| 0|[2003, hong, kong...|[2003, hong, kong...|(10000,[2,3,4,7,8...|[7693.0,848.0]|[0.90071420208406...| 0.0|
2003 world strike...| 0|[2003, world, str...|[2003, world, str...|(10000,[1,2,7,8,9...|[7693.0,848.0]|[0.90071420208406...| 0.0|
2007 hiv-1 human ...| 1|[2007, hiv-1, hum...|[2007, hiv-1, hum...|(10000,[0,1,2,3,4...|[748.0,9253.0]|[0.07479252074792...| 1.0|
2008 field experi...| 1|[2008, field, exp...|[2008, field, exp...|(10000,[1,2,3,4,5...| [535.0,712.0]|[0.42902967121090...| 1.0|
2009 h1n1 influen...| 0|[2009, h1n1, infl...|[2009, h1n1, infl...|(10000,[0,1,2,3,4...| [889.0,385.0]|[0.69780219780219...| 0.0|
2009 infection no...| 0|[2009, infection,...|[2009, infection,...|(10000,[0,1,2,5,7...|[7693.0,848.0]|[0.90071420208406...| 0.0|
2009 pandemic inf...| 1|[2009, pandemic, ...|[2009, pandemic, ...|(10000,[0,1,2,3,4...|[748.0,9253.0]|[0.07479252074792...| 1.0|
2010 increase num...| 1|[2010, increase, ...|[2010, increase, ...|(10000,[0,1,2,5,6...| [185.0,564.0]|[0.24699599465954...| 1.0|
+--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+----------+
only showing top 20 rows

In [27]:
from pyspark.sql.functions import col
see = predictions.select('y', col('y').alias('label'), 'rawPrediction')

In [28]:
see.show()

+---+-----+--------------+
 y|label| rawPrediction|
+---+-----+--------------+
 0| 0|[7693.0,848.0]|
 1| 1| [535.0,712.0]|
 1| 1|[748.0,9253.0]|
 1| 1|[748.0,9253.0]|
 1| 1|[748.0,9253.0]|
 1| 1|[748.0,9253.0]|
 1| 1| [185.0,564.0]|
 0| 0|[7693.0,848.0]|
 1| 1|[748.0,9253.0]|
 1| 1| [535.0,712.0]|
 0| 0|[7693.0,848.0]|
 1| 1|[748.0,9253.0]|
 0| 0|[7693.0,848.0]|
 0| 0|[7693.0,848.0]|
 1| 1|[748.0,9253.0]|
 1| 1| [535.0,712.0]|
 0| 0| [889.0,385.0]|
 0| 0|[7693.0,848.0]|
 1| 1|[748.0,9253.0]|
 1| 1| [185.0,564.0]|
+---+-----+--------------+
only showing top 20 rows

In [29]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(see, {evaluator.metricName: "areaUnderROC"})))

Test: Area Under ROC: 0.8246139516794767

In [30]:
print ("numNodes = ", dtModel.numNodes)
print ("depth = ", dtModel.depth)

numNodes = 13
depth = 3

In [31]:
# Implementing random forest classifier.

In [32]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 23269
Test Dataset Count: 9914

In [33]:
trainingData.show()

+--------------------+---+--------------------+--------------------+--------------------+
 processed_text| y| words| filtered| features|
+--------------------+---+--------------------+--------------------+--------------------+
1 deadline compli...| 0|[1, deadline, com...|[1, deadline, com...|(10000,[2,3,4,5,8...|
1 introduction tr...| 1|[1, introduction,...|[1, introduction,...|(10000,[0,1,2,5,6...|
1,4-glcnac- ␤ pre...| 1|[1,4-glcnac-, ␤, ...|[1,4-glcnac-, ␤, ...|(10000,[0,1,2,3,4...|
10 case 23.81 urt...| 0|[10, case, 23.81,...|[10, case, 23.81,...|(10000,[1,2,3,4,5...|
114 catalytic hyd...| 1|[114, catalytic, ...|[114, catalytic, ...|(10000,[0,1,2,5,6...|
120 year pass wal...| 1|[120, year, pass,...|[120, year, pass,...|(10000,[0,1,2,3,4...|
17 160 type modif...| 1|[17, 160, type, m...|[17, 160, type, m...|(10000,[0,2,3,4,6...|
170 million chron...| 0|[170, million, ch...|[170, million, ch...|(10000,[0,1,2,5,7...|
170 million peopl...| 1|[170, million, pe...|[170, million, pe...|(10000,[0,1,2,5,6...|
1918 world war wi...| 0|[1918, world, war...|[1918, world, war...|(10000,[1,2,3,4,5...|
1959 richard feyn...| 1|[1959, richard, f...|[1959, richard, f...|(10000,[0,1,2,3,4...|
1980 average new ...| 0|[1980, average, n...|[1980, average, n...|(10000,[1,2,5,6,7...|
1980s world healt...| 0|[1980s, world, he...|[1980s, world, he...|(10000,[1,2,5,6,7...|
1995 haemophilus ...| 1|[1995, haemophilu...|[1995, haemophilu...|(10000,[0,1,2,7,8...|
1997 highly patho...| 1|[1997, highly, pa...|[1997, highly, pa...|(10000,[0,1,2,3,4...|
1997 study report...| 1|[1997, study, rep...|[1997, study, rep...|(10000,[0,1,2,3,4...|
1998 world health...| 0|[1998, world, hea...|[1998, world, hea...|(10000,[2,5,7,8,9...|
20 year alphaviru...| 1|[20, year, alphav...|[20, year, alphav...|(10000,[0,1,2,3,4...|
200 different typ...| 1|[200, different, ...|[200, different, ...|(10000,[0,1,2,3,4...|
2000-year medicin...| 1|[2000-year, medic...|[2000-year, medic...|(10000,[0,1,2,3,4...|
+--------------------+---+--------------------+--------------------+--------------------+
only showing top 20 rows

In [34]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="y", \
                            featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [35]:
predictions = rfModel.transform(testData)

In [36]:
predictions.show()

+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 processed_text| y| words| filtered| features| rawPrediction| probability|prediction|
+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
100,000 patient a...| 0|[100,000, patient...|[100,000, patient...|(10000,[2,3,4,7,8...|[15.3258884959666...|[0.76629442479833...| 0.0|
14 species canadi...| 1|[14, species, can...|[14, species, can...|(10000,[1,2,5,7,8...|[5.96712663310291...|[0.29835633165514...| 1.0|
14.69 60.02 propo...| 1|[14.69, 60.02, pr...|[14.69, 60.02, pr...|(10000,[1,2,5,6,7...|[4.03148821189747...|[0.20157441059487...| 1.0|
15 year ago opden...| 1|[15, year, ago, o...|[15, year, ago, o...|(10000,[0,1,2,6,7...|[6.44557733414162...|[0.32227886670708...| 1.0|
1970s tomisaku ka...| 1|[1970s, tomisaku,...|[1970s, tomisaku,...|(10000,[0,2,3,4,5...|[6.92717897396851...|[0.34635894869842...| 1.0|
1990 computer use...| 1|[1990, computer, ...|[1990, computer, ...|(10000,[0,2,6,7,8...|[7.1075313263834,...|[0.35537656631917...| 1.0|
1992 laboratory p...| 1|[1992, laboratory...|[1992, laboratory...|(10000,[0,1,2,5,6...|[7.20195240999851...|[0.36009762049992...| 1.0|
1995 ﻿1 pilot iss...| 0|[1995, ﻿1, pilot,...|[1995, ﻿1, pilot,...|(10000,[3,4,5,8,9...|[14.9737191261008...|[0.74868595630504...| 0.0|
2 billion people ...| 1|[2, billion, peop...|[2, billion, peop...|(10000,[0,1,2,3,4...|[2.42822816460086...|[0.12141140823004...| 1.0|
2 million people ...| 1|[2, million, peop...|[2, million, peop...|(10000,[0,1,2,5,7...|[6.53415859875167...|[0.32670792993758...| 1.0|
2003 2 million ca...| 0|[2003, 2, million...|[2003, 2, million...|(10000,[1,2,5,7,8...|[16.4069184845582...|[0.82034592422791...| 0.0|
2003 coronavirus ...| 1|[2003, coronaviru...|[2003, coronaviru...|(10000,[0,1,2,3,4...|[3.21062834196916...|[0.16053141709845...| 1.0|
2003 hong kong pr...| 0|[2003, hong, kong...|[2003, hong, kong...|(10000,[2,3,4,7,8...|[15.3207292874978...|[0.76603646437489...| 0.0|
2003 world strike...| 0|[2003, world, str...|[2003, world, str...|(10000,[1,2,7,8,9...|[16.6708461426496...|[0.83354230713248...| 0.0|
2007 hiv-1 human ...| 1|[2007, hiv-1, hum...|[2007, hiv-1, hum...|(10000,[0,1,2,3,4...|[6.30864918022533...|[0.31543245901126...| 1.0|
2008 field experi...| 1|[2008, field, exp...|[2008, field, exp...|(10000,[1,2,3,4,5...|[9.58464177642491...|[0.47923208882124...| 1.0|
2009 h1n1 influen...| 0|[2009, h1n1, infl...|[2009, h1n1, infl...|(10000,[0,1,2,3,4...|[14.9041821293565...|[0.74520910646782...| 0.0|
2009 infection no...| 0|[2009, infection,...|[2009, infection,...|(10000,[0,1,2,5,7...|[15.5356874828787...|[0.77678437414393...| 0.0|
2009 pandemic inf...| 1|[2009, pandemic, ...|[2009, pandemic, ...|(10000,[0,1,2,3,4...|[2.98836085159499...|[0.14941804257974...| 1.0|
2010 increase num...| 1|[2010, increase, ...|[2010, increase, ...|(10000,[0,1,2,5,6...|[5.51395286842580...|[0.27569764342129...| 1.0|
+--------------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [37]:
from pyspark.sql.functions import col
see = predictions.select('y', col('y').alias('label'), 'rawPrediction')

In [38]:
see.show()

+---+-----+--------------------+
 y|label| rawPrediction|
+---+-----+--------------------+
 0| 0|[15.3258884959666...|
 1| 1|[5.96712663310291...|
 1| 1|[4.03148821189747...|
 1| 1|[6.44557733414162...|
 1| 1|[6.92717897396851...|
 1| 1|[7.1075313263834,...|
 1| 1|[7.20195240999851...|
 0| 0|[14.9737191261008...|
 1| 1|[2.42822816460086...|
 1| 1|[6.53415859875167...|
 0| 0|[16.4069184845582...|
 1| 1|[3.21062834196916...|
 0| 0|[15.3207292874978...|
 0| 0|[16.6708461426496...|
 1| 1|[6.30864918022533...|
 1| 1|[9.58464177642491...|
 0| 0|[14.9041821293565...|
 0| 0|[15.5356874828787...|
 1| 1|[2.98836085159499...|
 1| 1|[5.51395286842580...|
+---+-----+--------------------+
only showing top 20 rows

In [39]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(see, {evaluator.metricName: "areaUnderROC"})))

Test: Area Under ROC: 0.9852079474398303

In [40]:
# Implementing Naive Bayes Algorithm

In [41]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 23269
Test Dataset Count: 9914

In [42]:
trainingData.show()

+--------------------+---+--------------------+--------------------+--------------------+
 processed_text| y| words| filtered| features|
+--------------------+---+--------------------+--------------------+--------------------+
1 deadline compli...| 0|[1, deadline, com...|[1, deadline, com...|(10000,[2,3,4,5,8...|
1 introduction tr...| 1|[1, introduction,...|[1, introduction,...|(10000,[0,1,2,5,6...|
1,4-glcnac- ␤ pre...| 1|[1,4-glcnac-, ␤, ...|[1,4-glcnac-, ␤, ...|(10000,[0,1,2,3,4...|
10 case 23.81 urt...| 0|[10, case, 23.81,...|[10, case, 23.81,...|(10000,[1,2,3,4,5...|
114 catalytic hyd...| 1|[114, catalytic, ...|[114, catalytic, ...|(10000,[0,1,2,5,6...|
120 year pass wal...| 1|[120, year, pass,...|[120, year, pass,...|(10000,[0,1,2,3,4...|
17 160 type modif...| 1|[17, 160, type, m...|[17, 160, type, m...|(10000,[0,2,3,4,6...|
170 million chron...| 0|[170, million, ch...|[170, million, ch...|(10000,[0,1,2,5,7...|
170 million peopl...| 1|[170, million, pe...|[170, million, pe...|(10000,[0,1,2,5,6...|
1918 world war wi...| 0|[1918, world, war...|[1918, world, war...|(10000,[1,2,3,4,5...|
1959 richard feyn...| 1|[1959, richard, f...|[1959, richard, f...|(10000,[0,1,2,3,4...|
1980 average new ...| 0|[1980, average, n...|[1980, average, n...|(10000,[1,2,5,6,7...|
1980s world healt...| 0|[1980s, world, he...|[1980s, world, he...|(10000,[1,2,5,6,7...|
1995 haemophilus ...| 1|[1995, haemophilu...|[1995, haemophilu...|(10000,[0,1,2,7,8...|
1997 highly patho...| 1|[1997, highly, pa...|[1997, highly, pa...|(10000,[0,1,2,3,4...|
1997 study report...| 1|[1997, study, rep...|[1997, study, rep...|(10000,[0,1,2,3,4...|
1998 world health...| 0|[1998, world, hea...|[1998, world, hea...|(10000,[2,5,7,8,9...|
20 year alphaviru...| 1|[20, year, alphav...|[20, year, alphav...|(10000,[0,1,2,3,4...|
200 different typ...| 1|[200, different, ...|[200, different, ...|(10000,[0,1,2,3,4...|
2000-year medicin...| 1|[2000-year, medic...|[2000-year, medic...|(10000,[0,1,2,3,4...|
+--------------------+---+--------------------+--------------------+--------------------+
only showing top 20 rows

In [43]:
from pyspark.ml.classification import NaiveBayes

# create the trainer and set its parameters
nb = NaiveBayes(labelCol="y", smoothing=1, modelType="multinomial")

# train the model
model = nb.fit(trainingData)

In [44]:
predictions = model.transform(testData)

In [45]:
from pyspark.sql.functions import col
see = predictions.select('y', col('y').alias('label'), 'rawPrediction')

In [46]:
see.show()

+---+-----+--------------------+
 y|label| rawPrediction|
+---+-----+--------------------+
 0| 0|[-18666.460436596...|
 1| 1|[-22603.981148977...|
 1| 1|[-22899.388092472...|
 1| 1|[-25924.598706984...|
 1| 1|[-17133.734486037...|
 1| 1|[-14173.444250163...|
 1| 1|[-9184.8233511717...|
 0| 0|[-2873.8723871638...|
 1| 1|[-27673.176842173...|
 1| 1|[-19191.953094462...|
 0| 0|[-24145.664466697...|
 1| 1|[-26097.160151904...|
 0| 0|[-15426.634830047...|
 0| 0|[-21017.088992637...|
 1| 1|[-17604.991833846...|
 1| 1|[-21349.718743404...|
 0| 0|[-15794.014598697...|
 0| 0|[-7266.3274231718...|
 1| 1|[-20257.909321387...|
 1| 1|[-24089.690589153...|
+---+-----+--------------------+
only showing top 20 rows

In [47]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(see, {evaluator.metricName: "areaUnderROC"})))

Test: Area Under ROC: 0.38929004266271305

In [48]:
# Implementing cross validation

In [49]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 23269
Test Dataset Count: 9914

In [50]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="y", maxIter=20, regParam=0.3, elasticNetParam=0, family = "binomial")
# lr = LogisticRegression()

In [51]:
from pyspark.sql.functions import col
updatedTrainingData = trainingData.select('y', col('y').alias('label'), 'features')

In [52]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=3)

# Run cross validations
cvModel = cv.fit(updatedTrainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [53]:
from pyspark.sql.functions import col
updatedTestData = testData.select('y', col('y').alias('label'), 'features')

In [54]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(updatedTestData)

In [55]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("Test: Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test: Area Under ROC: 0.9956686209314644

In [56]:
evaluator.evaluate(predictions)

Out[109]: 0.9956686209314627

In [57]:
predictions.show()

+---+-----+--------------------+--------------------+--------------------+----------+
 y|label| features| rawPrediction| probability|prediction|
+---+-----+--------------------+--------------------+--------------------+----------+
 0| 0|(10000,[2,3,4,7,8...|[8.02256287819271...|[0.99967212915841...| 0.0|
 1| 1|(10000,[1,2,5,7,8...|[-4.8676098316721...|[0.00763301677090...| 1.0|
 1| 1|(10000,[1,2,5,6,7...|[-6.9189511334220...|[9.87889694787093...| 1.0|
 1| 1|(10000,[0,1,2,6,7...|[-5.9910607788866...|[0.00249477018013...| 1.0|
 1| 1|(10000,[0,2,3,4,5...|[-1.6245902541570...|[0.16457279053114...| 1.0|
 1| 1|(10000,[0,2,6,7,8...|[-3.0891965829182...|[0.04355509146977...| 1.0|
 1| 1|(10000,[0,1,2,5,6...|[-1.0486892781689...|[0.25947687480704...| 1.0|
 0| 0|(10000,[3,4,5,8,9...|[1.69920859487066...|[0.84543134459088...| 0.0|
 1| 1|(10000,[0,1,2,3,4...|[-8.4287966324691...|[2.18436524172421...| 1.0|
 1| 1|(10000,[0,1,2,5,7...|[-3.0981930889562...|[0.04318184981257...| 1.0|
 0| 0|(10000,[1,2,5,7,8...|[10.6581510023021...|[0.99997649211145...| 0.0|
 1| 1|(10000,[0,1,2,3,4...|[-5.4401085591703...|[0.00432026651645...| 1.0|
 0| 0|(10000,[2,3,4,7,8...|[5.67047749554621...|[0.99656561631066...| 0.0|
 0| 0|(10000,[1,2,7,8,9...|[9.11526744241215...|[0.99989003818736...| 0.0|
 1| 1|(10000,[0,1,2,3,4...|[-6.9487283032745...|[9.58934584190184...| 1.0|
 1| 1|(10000,[1,2,3,4,5...|[-3.9395888625760...|[0.01908489255732...| 1.0|
 0| 0|(10000,[0,1,2,3,4...|[6.20364835789404...|[0.99798204035820...| 0.0|
 0| 0|(10000,[0,1,2,5,7...|[4.00871533890843...|[0.98216708157381...| 0.0|
 1| 1|(10000,[0,1,2,3,4...|[-5.8360252895251...|[0.00291192338001...| 1.0|
 1| 1|(10000,[0,1,2,5,6...|[-5.7692602385461...|[0.00311234928372...| 1.0|
+---+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [58]:
from pyspark.sql.functions import col
updatedPredictions = predictions.select('prediction', col('prediction').alias('ctx'), 'label')